# Toronto Neighborhoods

- imports

In [1]:
from bs4 import BeautifulSoup
import requests
import numpy as np
import pandas as pd
from geopy.geocoders import Nominatim
import folium
import json
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors

- will use 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
- check for status code and headers

In [2]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
result = requests.get(url)
print(url)
print(result.status_code)
print(result.headers)

https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M
200
{'Date': 'Fri, 26 Jun 2020 09:03:04 GMT', 'Vary': 'Accept-Encoding,Cookie,Authorization', 'Server': 'ATS/8.0.8', 'X-Content-Type-Options': 'nosniff', 'P3p': 'CP="See https://en.wikipedia.org/wiki/Special:CentralAutoLogin/P3P for more info."', 'Content-Language': 'en', 'Last-Modified': 'Thu, 18 Jun 2020 21:29:18 GMT', 'Content-Type': 'text/html; charset=UTF-8', 'Content-Encoding': 'gzip', 'Age': '22464', 'X-Cache': 'cp3064 hit, cp3062 hit/9', 'X-Cache-Status': 'hit-front', 'Server-Timing': 'cache;desc="hit-front"', 'Strict-Transport-Security': 'max-age=106384710; includeSubDomains; preload', 'Set-Cookie': 'WMF-Last-Access=26-Jun-2020;Path=/;HttpOnly;secure;Expires=Tue, 28 Jul 2020 12:00:00 GMT, WMF-Last-Access-Global=26-Jun-2020;Path=/;Domain=.wikipedia.org;HttpOnly;secure;Expires=Tue, 28 Jul 2020 12:00:00 GMT, GeoIP=GB:ENG:Streatham:51.42:-0.12:v4; Path=/; secure; Domain=.wikipedia.org', 'X-Client-IP': '78.149.69.38', 

- will get data + clean it

In [3]:
soup = BeautifulSoup(result.content, 'html.parser')
table = soup.find('table')
trs = table.find_all('tr')
rows = []
for tr in trs:
    i = tr.find_all('td')
    if i:
        rows.append(i)
        
lst = []
for row in rows:
    postalcode = row[0].text.rstrip()
    borough = row[1].text.rstrip()
    neighborhood = row[2].text.rstrip()
    if borough != 'Not assigned':
        if neighborhood == 'Not assigned':
            neighborhood = borough
        lst.append([postalcode, borough, neighborhood])

# lst

- convert into a dataframe

In [4]:
cols = ['PostalCode', 'Borough', 'Neighborhood']
df = pd.DataFrame(lst, columns=cols)
print(df.shape)
# df[df.duplicated(['PostalCode'], keep=False)] - this would have shown the duplicate PostalCodes

(103, 3)


- custom groupby / agg to merge Neighborhoods
    - groupby PostalCode, keep the first Borough and join() Neighborhoods

In [5]:
df = df.groupby('PostalCode').agg(
    {
        'Borough':'first', 
        'Neighborhood': ', '.join,}
    ).reset_index()

- will check if 'M5A' example is done correctly

In [6]:
df.loc[df['PostalCode'] == 'M5A']

,PostalCode,Borough,Neighborhood
53,M5A,Downtown Toronto,"Regent Park, Harbourfront"


- will check the df.shape

In [7]:
df.shape

(103, 3)

- will read geo data from a CSV
- need to rename a column so the merge (below) will work

In [8]:
dfgeo = pd.read_csv("data/Geospatial_Coordinates.csv")
dfgeo.rename(columns={'Postal Code': 'PostalCode'}, inplace=True)

- need to merge 2 dataframes based on a PostalCode column

In [9]:
df2 = pd.merge(df, dfgeo, on="PostalCode", how='left')

- let's check if the example returns the expected value(s)

In [10]:
df2.loc[df2['PostalCode'] == 'M5G']

,PostalCode,Borough,Neighborhood,Latitude,Longitude
57,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383


In [11]:
df2.loc[df2['PostalCode'] == 'M9V']

,PostalCode,Borough,Neighborhood,Latitude,Longitude
101,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest...",43.739416,-79.588437


- Toronto map

In [12]:
geolocator = Nominatim(user_agent="coursera")
address = 'Toronto'
try:
    location = geolocator.geocode(address)
    latitude = location.latitude
    longitude = location.longitude
    print('The geograpical coordinates of {} are {}, {}.'.format(address, latitude, longitude))
except AttributeError:
    print('Cannot find: {}, will drop index: {}'.format(address, index))

my_map = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(df2['Latitude'], df2['Longitude'], df2['PostalCode']):
    label = folium.Popup(label)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(my_map)  
    
my_map

The geograpical coordinates of Toronto are 43.6534817, -79.3839347.


- Will analyze with 4Square

- will focus on Etobicoke

In [14]:
df2.set_index('PostalCode', inplace = True) 
neighborhood_latitude = df2.loc['M9V']['Latitude']
neighborhood_longitude = df2.loc['M9V']['Longitude']

In [15]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=IHEEOGCRRWIVMO4AU0BC4AYXXJ3VJWXCTGW4DMD0HRALYQY1&client_secret=SUIK3UFTXEE01DSPHGP5HAU1Y4C15C4ND0B2ZYV2KALSJY4D&v=20180605&ll=43.739416399999996,-79.5884369&radius=500&limit=100'

In [16]:
results = requests.get(url).json()
# results

In [17]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [18]:
venues = results['response']['groups'][0]['items']

In [19]:
results

{'meta': {'code': 200, 'requestId': '5ef61264564b600a1266e898'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Rexdale',
  'headerFullLocation': 'Rexdale, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 9,
  'suggestedBounds': {'ne': {'lat': 43.7439164045, 'lng': -79.58222007762089},
   'sw': {'lat': 43.734916395499994, 'lng': -79.59465372237912}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4be58dc4cf200f479154133c',
       'name': 'Shoppers Drug Mart',
       'location': {'address': '1530 Albion Rd',
        'crossStreet': 'Albion Mall',
        'lat': 43.741685,
        'lng': -79.584487,
        'labeledLatLngs': [{'label': 'display',
          

In [20]:
nearby_venues = json_normalize(venues) # flatten JSON
nearby_venues

,reasons.count,reasons.items,referralId,venue.categories,venue.id,venue.location.address,venue.location.cc,venue.location.city,venue.location.country,venue.location.crossStreet,venue.location.distance,venue.location.formattedAddress,venue.location.labeledLatLngs,venue.location.lat,venue.location.lng,venue.location.postalCode,venue.location.state,venue.name,venue.photos.count,venue.photos.groups
0,0,"[{'summary': 'This spot is popular', 'type': '...",e-0-4be58dc4cf200f479154133c-0,"[{'id': '4bf58dd8d48988d10f951735', 'name': 'P...",4be58dc4cf200f479154133c,1530 Albion Rd,CA,Etobicoke,Canada,Albion Mall,405,"[1530 Albion Rd (Albion Mall), Etobicoke ON M9...","[{'label': 'display', 'lat': 43.741685, 'lng':...",43.741685,-79.584487,M9V 1B4,ON,Shoppers Drug Mart,0,[]
1,0,"[{'summary': 'This spot is popular', 'type': '...",e-0-4c633939e1621b8d48842553-1,"[{'id': '4bf58dd8d48988d1c5941735', 'name': 'S...",4c633939e1621b8d48842553,"6210 Finch Ave West, Store 103",CA,Toronto,Canada,at Albion Rd.,372,"[6210 Finch Ave West, Store 103 (at Albion Rd....","[{'label': 'display', 'lat': 43.74264512142215...",43.742645,-79.589643,M9V 0A1,ON,Subway,0,[]
2,0,"[{'summary': 'This spot is popular', 'type': '...",e-0-4be70e26cf200f47e334153c-2,"[{'id': '4d4ae6fc7a7b7dea34424761', 'name': 'F...",4be70e26cf200f47e334153c,1530 Albion Rd.,CA,Etobicoke,Canada,at Kipling Ave. (Albion Centre),370,[1530 Albion Rd. (at Kipling Ave. (Albion Cent...,"[{'label': 'display', 'lat': 43.74120165509377...",43.741202,-79.584545,M9V 1B4,ON,Popeyes Louisiana Kitchen,0,[]
3,0,"[{'summary': 'This spot is popular', 'type': '...",e-0-4cd4738cdfb4a1cd4337535c-3,"[{'id': '5370f356bcbc57f1066c94c2', 'name': 'B...",4cd4738cdfb4a1cd4337535c,1530 Albion Rd,CA,Etobicoke,Canada,Near Finch Ave. W.,413,"[1530 Albion Rd (Near Finch Ave. W.), Etobicok...","[{'label': 'display', 'lat': 43.7416936, 'lng'...",43.741694,-79.584373,M9V 1B4,ON,The Beer Store,0,[]
4,0,"[{'summary': 'This spot is popular', 'type': '...",e-0-5112b872e4b0c0a78d7bcd27-4,"[{'id': '4bf58dd8d48988d118951735', 'name': 'G...",5112b872e4b0c0a78d7bcd27,1620 Albion road,CA,Toronto,Canada,Albion Road and Finch Ave,319,"[1620 Albion road (Albion Road and Finch Ave),...","[{'label': 'display', 'lat': 43.74184023292111...",43.741840,-79.590561,NaN,ON,Sunny Foodmart,0,[]
5,0,"[{'summary': 'This spot is popular', 'type': '...",e-0-4d8ba6910c4e41bdaaf7667f-5,"[{'id': '4bf58dd8d48988d1ca941735', 'name': 'P...",4d8ba6910c4e41bdaaf7667f,"1530 Albion Road, Unit T25",CA,Etobicoke,Canada,NaN,397,"[1530 Albion Road, Unit T25, Etobicoke ON M9V ...","[{'label': 'display', 'lat': 43.74156896801906...",43.741569,-79.584489,M9V 1B4,ON,Pizza Pizza,0,[]
6,0,"[{'summary': 'This spot is popular', 'type': '...",e-0-4b04a05bf964a520c45522e3-6,"[{'id': '4bf58dd8d48988d118951735', 'name': 'G...",4b04a05bf964a520c45522e3,1530 Albion Rd,CA,Etobicoke,Canada,at Finch Ave. W.,413,"[1530 Albion Rd (at Finch Ave. W.), Etobicoke ...","[{'label': 'display', 'lat': 43.741696, 'lng':...",43.741696,-79.584379,M9V 1B4,ON,Sheriff's No Frills,0,[]
7,0,"[{'summary': 'This spot is popular', 'type': '...",e-0-4c1951d6834e2d7f2d3a2a80-7,"[{'id': '4bf58dd8d48988d16e941735', 'name': 'F...",4c1951d6834e2d7f2d3a2a80,"1530 Albion Road, Unit F-1",CA,Toronto,Canada,NaN,427,"[1530 Albion Road, Unit F-1, Toronto ON M9V 1B...","[{'label': 'display', 'lat': 43.7417571, 'lng'...",43.741757,-79.584230,M9V 1B4,ON,McDonald's,0,[]
8,0,"[{'summary': 'This spot is popular', 'type': '...",e-0-4da73b855da37feee89a1143-8,"[{'id': '4bf58dd8d48988d1ca941735', 'name': 'P...",4da73b855da37feee89a1143,1701 Martingrove Road,CA,Toronto,Canada,NaN,315,"[1701 Martingrove Road, Toronto ON M9V 4N4, Ca...","[{'label': 'display', 'lat': 43.73676056243852...",43.736761,-79.589817,M9V 4N4,ON,Pizza Nova,0,[]


In [21]:
# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues = nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Shoppers Drug Mart,Pharmacy,43.741685,-79.584487
1,Subway,Sandwich Place,43.742645,-79.589643
2,Popeyes Louisiana Kitchen,Fried Chicken Joint,43.741202,-79.584545
3,The Beer Store,Beer Store,43.741694,-79.584373
4,Sunny Foodmart,Grocery Store,43.741840,-79.590561


In [22]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

9 venues were returned by Foursquare.


In [23]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET,  
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [24]:
df2

,Borough,Neighborhood,Latitude,Longitude
PostalCode,,,,
M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
M1G,Scarborough,Woburn,43.770992,-79.216917
M1H,Scarborough,Cedarbrae,43.773136,-79.239476
M1J,Scarborough,Scarborough Village,43.744734,-79.239476
M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park",43.727929,-79.262029
M1L,Scarborough,"Golden Mile, Clairlea, Oakridge",43.711112,-79.284577
M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West",43.716316,-79.239476


In [25]:
venues = getNearbyVenues(names=df2['Borough'],latitudes=df2['Latitude'],longitudes=df2['Longitude'])

Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
East York
East York
East Toronto
East York
East York
East York
East Toronto
East Toronto
East Toronto
Central Toronto
Central Toronto
Central Toronto
Central Toronto
Central Toronto
Central Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
North York
Central Toronto
Central Toronto
Central Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
North York
North York
York
York
Downtown Toronto
Wes

In [26]:
print(venues.shape)
venues.head()

(2130, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Scarborough,43.806686,-79.194353,Wendy’s,43.807448,-79.199056,Fast Food Restaurant
1,Scarborough,43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
2,Scarborough,43.763573,-79.188711,RBC Royal Bank,43.766790,-79.191151,Bank
3,Scarborough,43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store
4,Scarborough,43.763573,-79.188711,Big Bite Burrito,43.766299,-79.190720,Mexican Restaurant


In [27]:
venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Central Toronto,113,113,113,113,113,113
Downtown Toronto,1218,1218,1218,1218,1218,1218
East Toronto,128,128,128,128,128,128
East York,74,74,74,74,74,74
Etobicoke,69,69,69,69,69,69
Mississauga,13,13,13,13,13,13
North York,243,243,243,243,243,243
Scarborough,93,93,93,93,93,93
West Toronto,161,161,161,161,161,161


In [28]:
print('There are {} unique categories.'.format(len(venues['Venue Category'].unique())))

There are 272 unique categories.


In [29]:
# one hot encoding
onehot = pd.get_dummies(venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
onehot['Neighborhood'] = venues['Neighborhood'] 

onehot.head()

,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [30]:
onehot.shape

(2130, 272)

In [31]:
grouped = onehot.groupby('Neighborhood').mean().reset_index()
grouped

,Neighborhood,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Central Toronto,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.008850,...,0.000000,0.008850,0.000000,0.008850,0.000000,0.000000,0.000000,0.000000,0.000000,0.008850
1,Downtown Toronto,0.00000,0.000821,0.000821,0.000821,0.000821,0.001642,0.002463,0.001642,0.013957,...,0.002463,0.011494,0.001642,0.003284,0.000000,0.006568,0.000821,0.000000,0.000821,0.005747
2,East Toronto,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.023438,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.007812,0.000000,0.000000,0.000000,0.015625
3,East York,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.013514,0.000000,0.000000,0.000000,0.000000,0.013514
4,Etobicoke,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.014493,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.014493,0.000000,0.000000
5,Mississauga,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.076923,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,North York,0.00823,0.000000,0.004115,0.000000,0.000000,0.000000,0.000000,0.000000,0.008230,...,0.000000,0.000000,0.004115,0.012346,0.000000,0.000000,0.000000,0.000000,0.008230,0.000000
7,Scarborough,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.010753,...,0.010753,0.000000,0.000000,0.010753,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,West Toronto,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.018634,0.000000,0.012422,0.000000,0.006211,0.000000,0.000000,0.000000,0.018634
9,York,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.055556,0.000000


In [32]:
grouped.shape

(10, 272)

In [33]:
num_top_venues = 5

for hood in grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = grouped[grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Central Toronto----
            venue  freq
0     Coffee Shop  0.07
1  Sandwich Place  0.06
2            Park  0.06
3            Café  0.05
4     Pizza Place  0.05


----Downtown Toronto----
         venue  freq
0  Coffee Shop  0.10
1         Café  0.06
2   Restaurant  0.04
3        Hotel  0.03
4       Bakery  0.02


----East Toronto----
                venue  freq
0    Greek Restaurant  0.06
1         Coffee Shop  0.05
2                Café  0.04
3             Brewery  0.04
4  Italian Restaurant  0.04


----East York----
            venue  freq
0     Coffee Shop  0.05
1            Bank  0.05
2            Park  0.05
3    Burger Joint  0.04
4  Sandwich Place  0.04


----Etobicoke----
                  venue  freq
0           Pizza Place  0.12
1        Sandwich Place  0.07
2                   Gym  0.04
3  Fast Food Restaurant  0.04
4           Coffee Shop  0.04


----Mississauga----
                       venue  freq
0                Coffee Shop  0.15
1                      Hotel  0.

In [34]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [35]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = grouped['Neighborhood']

for ind in np.arange(grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Central Toronto,Coffee Shop,Sandwich Place,Park,Café,Pizza Place,Restaurant,Sushi Restaurant,Dessert Shop,Clothing Store,Gym
1,Downtown Toronto,Coffee Shop,Café,Restaurant,Hotel,Japanese Restaurant,Italian Restaurant,Bakery,Park,Gym,Beer Bar
2,East Toronto,Greek Restaurant,Coffee Shop,Italian Restaurant,Brewery,Café,Pizza Place,Park,Restaurant,Ice Cream Shop,American Restaurant
3,East York,Bank,Park,Coffee Shop,Sandwich Place,Pizza Place,Pharmacy,Burger Joint,Fast Food Restaurant,Restaurant,Pet Store
4,Etobicoke,Pizza Place,Sandwich Place,Gym,Coffee Shop,Pharmacy,Grocery Store,Fast Food Restaurant,Beer Store,Fried Chicken Joint,Pool
5,Mississauga,Hotel,Coffee Shop,Gym,American Restaurant,Fried Chicken Joint,Intersection,Burrito Place,Mediterranean Restaurant,Middle Eastern Restaurant,Gas Station
6,North York,Coffee Shop,Clothing Store,Restaurant,Bank,Sandwich Place,Pizza Place,Japanese Restaurant,Grocery Store,Fast Food Restaurant,Café
7,Scarborough,Chinese Restaurant,Breakfast Spot,Bank,Bakery,Coffee Shop,Intersection,Pizza Place,Fast Food Restaurant,Pharmacy,Grocery Store
8,West Toronto,Café,Bar,Coffee Shop,Restaurant,Italian Restaurant,Bakery,Diner,Breakfast Spot,Pizza Place,Park
9,York,Park,Convenience Store,Bakery,Discount Store,Hockey Arena,Playground,Field,Bar,Trail,Caribbean Restaurant


In [36]:
# set number of clusters
kclusters = 5

grouped_clustering = grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([2, 2, 2, 0, 4, 3, 2, 0, 2, 1])

In [37]:
merged = grouped

# add clustering labels
merged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
merged = merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

merged.head() # check the last columns!

,Neighborhood,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Central Toronto,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.008850,...,Coffee Shop,Sandwich Place,Park,Café,Pizza Place,Restaurant,Sushi Restaurant,Dessert Shop,Clothing Store,Gym
1,Downtown Toronto,0.0,0.000821,0.000821,0.000821,0.000821,0.001642,0.002463,0.001642,0.013957,...,Coffee Shop,Café,Restaurant,Hotel,Japanese Restaurant,Italian Restaurant,Bakery,Park,Gym,Beer Bar
2,East Toronto,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.023438,...,Greek Restaurant,Coffee Shop,Italian Restaurant,Brewery,Café,Pizza Place,Park,Restaurant,Ice Cream Shop,American Restaurant
3,East York,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,Bank,Park,Coffee Shop,Sandwich Place,Pizza Place,Pharmacy,Burger Joint,Fast Food Restaurant,Restaurant,Pet Store
4,Etobicoke,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.014493,...,Pizza Place,Sandwich Place,Gym,Coffee Shop,Pharmacy,Grocery Store,Fast Food Restaurant,Beer Store,Fried Chicken Joint,Pool
